### Custom functions

- **Incorporating Arbitrary Functions in LangChain Pipeline**:
  - LangChain allows the use of arbitrary functions within its pipeline.
  - A key requirement is that all inputs to these functions must be encapsulated in a SINGLE argument.
  - For functions that originally accept multiple arguments, a wrapper function should be created. This wrapper must accept a single input and then unpack it into the required multiple arguments.

In [1]:
from operator import itemgetter

from langchain.chat_models.openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda

In [2]:
def length_function(text):
  return len(text)

def _multiple_length_function(text1, text2):
  return len(text1) * len(text2)

def multiple_length_function(_dict):
  return _multiple_length_function(_dict["text1"], _dict["text2"])

In [3]:
prompt = ChatPromptTemplate.from_template("What is {a} + {b}")
model = ChatOpenAI()

In [4]:
chain1 = prompt | model

In [5]:
chain2 = (
    {
        "a": itemgetter("foo") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
)

In [6]:
chain2.invoke({"foo": "bar", "bar": "gah"})

AIMessage(content='3 + 9 equals 12.')

 **Usage of RunnableConfig with Runnable Lambdas**:
  - Runnable lambdas in LangChain can optionally incorporate a `RunnableConfig`.
  - This configuration is useful for passing additional details like callbacks, tags, and other configuration information to nested runs within the pipeline.


In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableConfig
import json

In [17]:
def parse_or_fix(text: str, config: RunnableConfig):
    fixing_chain = (
        ChatPromptTemplate.from_template(
            "Fix the following text:\n\n```text\n{input}\n```\nError: {error}"
            " Don't narrate, just respond with the fixed data."
        )
        | ChatOpenAI()
        | StrOutputParser()
    )

    for _ in range(3):
        try:
            return json.loads(text)
        except Exception as e:
            text = fixing_chain.invoke({"input": text, "error": e}, config)
    return "Failed to parse"

In [19]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    output = RunnableLambda(parse_or_fix).invoke(
        "{foo: bar}", {"tags": ["my-tag"], "callbacks": [cb]}
    )
    print(output)
    print(cb)

My precious.
Tokens Used: 54
	Prompt Tokens: 51
	Completion Tokens: 3
Successful Requests: 1
Total Cost (USD): $8.25e-05
